<a href="https://colab.research.google.com/github/kaifghori22/ML-/blob/main/Voice_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
Root = "/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio"
os.chdir(Root)

In [ ]:
ls

speech-emotion-recognition-ravdess-data/


In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
x_train

array([[-4.40141327e+02,  1.52982121e+01, -2.29905319e+01, ...,
         2.92085082e-04,  3.00491840e-04,  2.24592353e-04],
       [-5.44744385e+02,  7.57021484e+01, -7.10741949e+00, ...,
         2.04873431e-05,  1.28833863e-05,  1.45689773e-05],
       [-6.09416016e+02,  5.49188614e+01, -1.21941590e+00, ...,
         2.64643131e-05,  1.52848424e-05,  6.05608284e-06],
       ...,
       [-6.20390076e+02,  4.79046860e+01, -1.07318096e+01, ...,
         1.21914854e-05,  6.27341888e-06,  3.88543913e-06],
       [-5.03494232e+02,  1.91213112e+01, -1.31133127e+01, ...,
         2.97792023e-04,  1.04964252e-04,  9.26514404e-05],
       [-6.43201599e+02,  6.22693329e+01,  2.69707012e+00, ...,
         1.61272674e-05,  1.10278697e-05,  7.53134918e-06]])

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
y_pred

array(['fearful', 'disgust', 'disgust', 'calm', 'fearful', 'calm',
       'happy', 'fearful', 'disgust', 'calm', 'fearful', 'disgust',
       'calm', 'calm', 'fearful', 'disgust', 'calm', 'fearful', 'disgust',
       'disgust', 'happy', 'disgust', 'happy', 'disgust', 'calm', 'happy',
       'disgust', 'disgust', 'calm', 'calm', 'calm', 'fearful', 'calm',
       'fearful', 'disgust', 'fearful', 'calm', 'fearful', 'calm', 'calm',
       'disgust', 'fearful', 'calm', 'disgust', 'fearful', 'calm',
       'disgust', 'calm', 'calm', 'happy', 'disgust', 'fearful',
       'fearful', 'fearful', 'calm', 'disgust', 'disgust', 'calm',
       'fearful', 'calm', 'fearful', 'disgust', 'fearful', 'calm',
       'fearful', 'happy', 'calm', 'disgust', 'disgust', 'happy',
       'disgust', 'fearful', 'disgust', 'calm', 'disgust', 'disgust',
       'fearful', 'calm', 'calm', 'fearful', 'disgust', 'disgust', 'calm',
       'disgust', 'disgust', 'fearful', 'happy', 'calm', 'fearful',
       'disgust', 'fear

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 69.27%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.78181818, 0.6728972 , 0.73267327, 0.51515152])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,fearful
1,happy,disgust
2,calm,disgust
3,happy,calm
4,disgust,fearful
5,calm,calm
6,happy,happy
7,happy,fearful
8,disgust,disgust
9,happy,calm


In [ ]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [ ]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['fearful'], dtype='<U7')